# 고객 대출등급 분류 해커톤

* 정형 데이터
* 분류
* 금융
* Macro F1 score
* [고객 대출등급 분류 바로가기](https://dacon.io/competitions/official/236214/overview/description)

[설명]  
대출 고객과 관련된 데이터 분석을 통해 고객의 대출등급 예측하는 AI 모델을 개발해야 합니다.

[주제]  
고객의 대출등급을 예측하는 AI 알고리즘 개발

___
## 패키지 선언

In [49]:
# 추가로 사용하실 패키지를 자유롭게 선언해주세요.
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_style("whitegrid")

# 그래프 한글 폰트
# import matplotlib.pyplot as plt
# plt.rcParams['font.family'] ='Malgun Gothic'
# plt.rcParams['axes.unicode_minus'] =False
from matplotlib import rc
rc('font', family='AppleGothic')

## 데이터 로드


In [50]:
df = pd.read_csv('./train.csv')
df.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B


____
# EDA 와 Feature Engineering

## 데이터의 이해

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


In [52]:
df.describe()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
count,9.629400e+04,9.629400e+04,96294.000000,96294.000000,96294.000000,9.629400e+04,9.629400e+04,96294.000000,96294.000000
mean,1.830400e+07,9.392672e+07,19.379590,25.304827,0.345681,8.225035e+05,4.282282e+05,54.380584,0.005805
std,1.032908e+07,9.956871e+07,33.569559,12.088566,0.919119,1.027745e+06,4.402111e+05,1414.769218,0.079966
min,1.200000e+06,0.000000e+00,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.020000e+07,5.760000e+07,12.650000,17.000000,0.000000,3.075720e+05,1.346160e+05,0.000000,0.000000
50%,1.680000e+07,7.800000e+07,18.740000,24.000000,0.000000,5.976960e+05,2.870040e+05,0.000000,0.000000
75%,2.400000e+07,1.128000e+08,25.540000,32.000000,0.000000,1.055076e+06,5.702160e+05,0.000000,0.000000
max,4.200000e+07,1.080000e+10,9999.000000,169.000000,30.000000,4.195594e+07,5.653416e+06,75768.000000,4.000000


## 1-B. EDA

## Feature Engineering

### 결측치/이상치 탐색

In [53]:
# 결측치 갯수 확인

df.isnull().sum()

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

In [54]:
# 숫자 데이터가 아닌 데이터들 위주로 결측치로 볼 수 있는 값이 있는지 확인

df["주택소유상태"].value_counts()

주택소유상태
MORTGAGE    47934
RENT        37705
OWN         10654
ANY             1
Name: count, dtype: int64

In [55]:
df["주택소유상태"] = df["주택소유상태"].replace('ANY', np.nan)
df["주택소유상태"].value_counts()

주택소유상태
MORTGAGE    47934
RENT        37705
OWN         10654
Name: count, dtype: int64

In [56]:
df.isnull().sum()

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          1
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

In [57]:
df.dropna(axis=0, inplace=True)
df.isnull().sum()

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

- ANY를 결측치로 보고 삭제

In [58]:
df["대출목적"].value_counts()

대출목적
부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4724
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: count, dtype: int64

In [59]:
df["근로기간"].value_counts()

근로기간
10+ years    31584
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

- Unknown을 결측치로 보지 않겠음

In [60]:
df["대출기간"].value_counts()

대출기간
36 months    64478
60 months    31815
Name: count, dtype: int64

In [61]:
df["대출등급"].unique().tolist()

['C', 'B', 'A', 'D', 'F', 'G', 'E']

### one-hot encoding

- 주택소유상태
- 대출목적
- 근로기간
- 대출기간

In [62]:
ohe_list = ['주택소유상태', '대출목적', '대출기간']

df_ohe = pd.get_dummies(df, columns=ohe_list)
df_ohe

,ID,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가,대출기간_ 36 months,대출기간_ 60 months
0,TRAIN_00000,12480000,6 years,72000000,18.90,15,0,0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,TRAIN_00001,14400000,10+ years,130800000,22.33,21,0,373572,234060.0,0.0,...,False,False,False,False,False,False,True,False,False,True
2,TRAIN_00002,12000000,5 years,96000000,8.60,14,0,928644,151944.0,0.0,...,False,False,False,False,False,False,False,False,True,False
3,TRAIN_00003,14400000,8 years,132000000,15.09,15,0,325824,153108.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,TRAIN_00004,18000000,Unknown,71736000,25.39,19,0,228540,148956.0,0.0,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,10+ years,210000000,9.33,33,0,974580,492168.0,0.0,...,False,False,False,False,False,False,False,False,True,False
96290,TRAIN_96290,28800000,10+ years,132000000,5.16,25,0,583728,855084.0,0.0,...,False,False,False,False,False,False,True,False,False,True
96291,TRAIN_96291,14400000,1 year,84000000,11.24,22,0,1489128,241236.0,0.0,...,False,False,False,False,False,False,False,False,True,False
96292,TRAIN_96292,15600000,5 years,66330000,17.30,21,2,1378368,818076.0,0.0,...,False,False,False,False,False,False,False,False,True,False


In [63]:
df_ohe.columns.tolist()

['ID',
 '대출금액',
 '근로기간',
 '연간소득',
 '부채_대비_소득_비율',
 '총계좌수',
 '최근_2년간_연체_횟수',
 '총상환원금',
 '총상환이자',
 '총연체금액',
 '연체계좌수',
 '대출등급',
 '주택소유상태_MORTGAGE',
 '주택소유상태_OWN',
 '주택소유상태_RENT',
 '대출목적_기타',
 '대출목적_부채 통합',
 '대출목적_소규모 사업',
 '대출목적_신용 카드',
 '대출목적_의료',
 '대출목적_이사',
 '대출목적_자동차',
 '대출목적_재생 에너지',
 '대출목적_주요 구매',
 '대출목적_주택',
 '대출목적_주택 개선',
 '대출목적_휴가',
 '대출기간_ 36 months',
 '대출기간_ 60 months']

In [66]:
df = df_ohe.drop('근로기간', axis=1)
df.columns.tolist()

['ID',
 '대출금액',
 '연간소득',
 '부채_대비_소득_비율',
 '총계좌수',
 '최근_2년간_연체_횟수',
 '총상환원금',
 '총상환이자',
 '총연체금액',
 '연체계좌수',
 '대출등급',
 '주택소유상태_MORTGAGE',
 '주택소유상태_OWN',
 '주택소유상태_RENT',
 '대출목적_기타',
 '대출목적_부채 통합',
 '대출목적_소규모 사업',
 '대출목적_신용 카드',
 '대출목적_의료',
 '대출목적_이사',
 '대출목적_자동차',
 '대출목적_재생 에너지',
 '대출목적_주요 구매',
 '대출목적_주택',
 '대출목적_주택 개선',
 '대출목적_휴가',
 '대출기간_ 36 months',
 '대출기간_ 60 months']

- 근로기간 -> 칼럼 삭제    // 이유 공부하기

### 대출등급 label encoding

- A: 6
- B: 5
- C: 4
- D: 3
- E: 2
- F: 1
- G: 0

In [69]:
def label_encoder(x):
    if type(x) is int:
        return x
    return 6 - (ord(x)-65)    # ord(): char를 정수로 변형

df['대출등급'] = df['대출등급'].apply(lambda x: label_encoder(x)).astype(int)
df['대출등급']

0        4
1        5
2        6
3        4
4        5
        ..
96289    4
96290    2
96291    6
96292    3
96293    4
Name: 대출등급, Length: 96293, dtype: int64

In [73]:
df.columns.tolist()

['ID',
 '대출금액',
 '연간소득',
 '부채_대비_소득_비율',
 '총계좌수',
 '최근_2년간_연체_횟수',
 '총상환원금',
 '총상환이자',
 '총연체금액',
 '연체계좌수',
 '대출등급',
 '주택소유상태_MORTGAGE',
 '주택소유상태_OWN',
 '주택소유상태_RENT',
 '대출목적_기타',
 '대출목적_부채 통합',
 '대출목적_소규모 사업',
 '대출목적_신용 카드',
 '대출목적_의료',
 '대출목적_이사',
 '대출목적_자동차',
 '대출목적_재생 에너지',
 '대출목적_주요 구매',
 '대출목적_주택',
 '대출목적_주택 개선',
 '대출목적_휴가',
 '대출기간_ 36 months',
 '대출기간_ 60 months']

___
# 모델선택

In [77]:
# EDA를 통해 데이터셋에 어울리는 모델을 선택해주세요.

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

___
# 모델 훈련

## Features - Target column 분리하기

In [80]:
X = df.copy().drop(['ID', '대출등급'], axis=1)
X.head()

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,주택소유상태_MORTGAGE,...,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가,대출기간_ 36 months,대출기간_ 60 months
0,12480000,72000000,18.90,15,0,0,0.0,0.0,0.0,False,...,False,False,False,False,False,False,False,False,True,False
1,14400000,130800000,22.33,21,0,373572,234060.0,0.0,0.0,True,...,False,False,False,False,False,False,True,False,False,True
2,12000000,96000000,8.60,14,0,928644,151944.0,0.0,0.0,True,...,False,False,False,False,False,False,False,False,True,False
3,14400000,132000000,15.09,15,0,325824,153108.0,0.0,0.0,True,...,False,False,False,False,False,False,False,False,True,False
4,18000000,71736000,25.39,19,0,228540,148956.0,0.0,0.0,False,...,False,False,False,False,True,False,False,False,False,True


In [81]:
y = df['대출등급']
y

0        4
1        5
2        6
3        4
4        5
        ..
96289    4
96290    2
96291    6
96292    3
96293    4
Name: 대출등급, Length: 96293, dtype: int64

In [82]:
# train set, test set 분류

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
X_train.shape, y_train.shape

((77034, 26), (77034,))

In [84]:
X_test.shape, y_test.shape

((19259, 26), (19259,))

___
# 4. 미션 4) 하이퍼 파라미터 튜닝

In [20]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# SearchCV를 통해 하이퍼 파라미터를 튜닝해주세요.

___
# 5. 미션 5) 모델 평가 및 선택

* 해당 대회의 평가 방식은 Macro F1 score 입니다.

In [21]:
from sklearn.metrics import f1_score

In [22]:
# Macro F1 score
f1_score(y_true, y_pred, average='macro')

NameError: name 'y_true' is not defined